- Jose Pablo Ponce 19092
- Gabriel Quiroz

In [2]:
#Importar las librerías

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz

## Parte 1 – Ingeniería de características


## Exploración de datos


Revise la data y realice las operaciones necesarias para unificar los datasets y que el dataset final contenga el mensaje del correo y la etiqueta que indique si es SPAM o no.

In [35]:
df1 = pd.read_csv('./completeSpamAssassin.csv/completeSpamAssassin.csv')  
df2 = pd.read_csv('./enronSpamSubset.csv/enronSpamSubset.csv')  

In [36]:
print(df1.sample(n=5).head())


      Unnamed: 0                                               Body  Label
256          256  NEWSLETTER COACHÂ’INVEST Â– SEPTEMBRE 2002Coac...      1
461          461  New Account For: zzzz@spamassassin.taint.org \...      1
3638        3638  [followups to spambayes@python.org please, unl...      0
3313        3313  David LeBlanc wrote:\n> Errr... not to be peda...      0
6036        6036                                              empty      0


In [37]:
print(df2.sample(n=5).head())

      Unnamed: 0.1  Unnamed: 0  \
8259         25488       25488   
9213         17774       17774   
4005         10773       10773   
520          11939       11939   
4207         13147       13147   

                                                   Body  Label  
8259  Subject: re : nomination for gas flow / 11 / 0...      0  
9213  Subject: project inga recap / follow up\n reca...      0  
4005  Subject: toshiba 1 . 6 g / dvd - rw super mult...      1  
520   Subject: give her something to smile about\n m...      1  
4207  Subject: need a new car - save thousands !\n h...      1  


In [38]:
df1 = df1.drop(columns=['Unnamed: 0'])
df2 = df2.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
print(df1.columns)
print(df2.columns)

Index(['Body', 'Label'], dtype='object')
Index(['Body', 'Label'], dtype='object')


In [110]:
df_final = pd.concat([df1, df2])
df_final = df_final.rename(columns={"Label": "Spam"})
df_final.sample(n=5).head() 


,Body,Spam
8703,Subject: start date : 12 / 18 / 01 ; hourahead...,0
2626,Subject: losing your hair ? order generic xana...,1
3744,URL: http://scriptingnews.userland.com/backiss...,0
2254,"Subject: vicodin ' only $ 150 online , no doc ...",1
2707,Subject: order verification\n % rnd _ alt % rn...,1


## Preprocesamiento

Aplique las técnicas de pre – procesamiento de lenguaje natural que considere necesarias 
(conversión de minúsculas, mayúsculas, eliminación de acentos, expansión de contracciones, eliminación de stop
words, etc.)

In [111]:
#conversion a minusculas
df_final['Body'] = df_final['Body'].str.lower()


In [112]:
#remover acentos
df_final['Body'] = df_final['Body'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


In [113]:
#expandir contracciones
import contractions
df_final["Body"] = df_final["Body"].apply(lambda x: contractions.fix(str(x)))

In [90]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joopa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joopa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
#eliminar stop words
stop_words = nltk.corpus.stopwords.words('english')
df_final["Body"]= df_final["Body"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))


In [115]:
#eliminar caracteres especiales, numeros y simbolos
import re
def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos =False):
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron,'', texto)
    return texto
oracion = 'Hoy es el partido de fútbol a las 7:45 @$%&'

df_final["Body"] = df_final["Body"].apply(lambda x: removerCaracteresEspecialesNumerosSimbolos(x, True))


In [117]:
df_final.sample(n=5).head() 


,Body,Spam
8077,subject registration confirmation spinner com...,0
8320,subject supervisor evaluation forms kitchen l...,0
9917,subject fyi left kathy message monday ju...,0
5376,fri bitbitchmagnesiumnet wrote smegma happe...,0
9532,subject revised nom kcs resources daren kcs ...,0


## Representación de texto
